### Oracle SQL: GRANT and REVOKE

We will:
- Create a dedicated user `trainer_user`
- Demonstrate a **DDL privilege** (CREATE TABLE)
- Demonstrate a **DML privilege** (SELECT)

👉 Open two SQL*Plus sessions:
- One as `system`
- Another as `trainer_user/trainer123@XEPDB1` (After the user is created)

_Follow the instructions step by step._

#### Connect to Pluggable Database (PDB)
Ensure you are inside `XEPDB1` container before creating users.

In [ ]:
ALTER SESSION SET CONTAINER = XEPDB1;

In [ ]:
SHOW CON_NAME;

#### Create the user `trainer_user`
We create the user with password `trainer123`, grant login privilege, and allow tablespace usage.

In [ ]:
DROP USER trainer_user CASCADE;

In [ ]:
CREATE USER trainer_user IDENTIFIED BY trainer123;

In [ ]:
GRANT CREATE SESSION TO trainer_user;

In [ ]:
ALTER USER trainer_user QUOTA UNLIMITED ON USERS;

In [ ]:
ALTER USER trainer_user ACCOUNT UNLOCK;

#### Verify that the user exists
This confirms that the account is available in the current container.

In [ ]:
SELECT username, account_status, common 
FROM dba_users 
WHERE username = 'TRAINER_USER';

#### Login as `trainer_user`
From a new SQL*Plus session, connect as:


In [ ]:
sqlplus trainer_user/trainer123@localhost:1521/XEPDB1

Check the current user:

In [ ]:
SHOW USER;

#### DDL Privilege: CREATE TABLE

This shows how the `CREATE TABLE` system privilege works.

#### Without privilege
Login as `trainer_user` and try:

In [ ]:
CREATE TABLE demo_table(id NUMBER);

👉 Expect **ORA-01031: insufficient privileges**.

#### Grant the privilege
As `system`user run:

In [ ]:
GRANT CREATE TABLE TO trainer_user;

#### With privilege
Login as `trainer_user` again and retry:

In [ ]:
CREATE TABLE demo_table(id NUMBER);

👉 This should now succeed.

#### Revoke the privilege
As `system` user run:

In [ ]:
REVOKE CREATE TABLE FROM trainer_user;

### After revoke
As `trainer_user`, try again:

In [ ]:
CREATE TABLE demo_table2(id NUMBER);

👉 Expect **ORA-01031: insufficient privileges**

#### DML Privilege: SELECT

This shows how object-level privileges work on tables owned by another user (SYSTEM).

#### Create a sample table as SYSTEM user
Insert a row for demonstration.

In [ ]:
CREATE TABLE students (
    student_id NUMBER PRIMARY KEY,
    name VARCHAR2(50)
);

In [ ]:
INSERT INTO students VALUES (1, 'Arjun');

In [ ]:
COMMIT;

#### Without privilege
As `trainer_user`, try:

In [ ]:
SELECT * FROM system.students;

👉 Expect **ORA-00942: table or view does not exist** (no access).

#### Grant SELECT privilege
As `system` user run:

In [ ]:
GRANT SELECT ON students TO trainer_user;

#### With privilege
As `trainer_user`, retry:

In [ ]:
SELECT * FROM system.students;

👉 Now the query should succeed and return results

### Try to INSERT

In [ ]:
INSERT INTO system.students VALUES(2, 'Ravi')

👉 Expect **ORA-01031: insufficient privileges**

#### Grant INSERT privilege
As `system` user run:

In [ ]:
GRANT INSERT ON students TO trainer_user;

#### With privilege
As `trainer_user`, retry:

In [ ]:
INSERT INTO system.students VALUES(2, 'Ravi')

In [ ]:
SELECT * FROM system.students;

#### Revoke the privilege
As `system` run:

In [ ]:
REVOKE SELECT ON students FROM trainer_user;

#### After revoke
As `trainer_user`, retry again:

In [ ]:
SELECT * FROM system.students;

## Conclusion
- **System privileges (DDL)** such as `CREATE TABLE` control what users can do in their own schema.
- **Object privileges (DML)** such as `SELECT` control access to objects owned by other users.
- **GRANT** gives a privilege, **REVOKE** removes it.